# 准备

In [2]:
import numpy as np
import pandas as pd
from pandas import option_context

# 一、读取数据

>读取CSV文件，`titles.csv`命名为`original_titles`，`credits.csv`命名为`original_credits`

In [3]:
original_titles=pd.read_csv('titles.csv')
original_credits=pd.read_csv('credits.csv')

In [4]:
cleaned_titles=original_titles.copy()
cleaned_credits=original_credits.copy()

# 二、评估数据和清理数据

在这一部分中，我们将对在上一部分建立的original_titles及original_creditsDataFrame所包含的数据进行评估\
主要从两个方面进行：结构和内容，即整齐度和干净度。\
数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；\
数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

>为了区分开经过清理的数据和原始的数据，我们创建新的变量cleaned_titles，让它为original_titles复制出的副本，以及创建新的变量cleaned_credits，让它为original_credits复制出的变量。

In [5]:
cleaned_titles=original_titles.copy()
cleaned_credits=original_credits.copy()

### 1.结构

In [6]:
original_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
2839,tm346334,Out of Thin Air,MOVIE,"Set within the stark Icelandic landscape, OUT ...",2017,NaN,85,"['crime', 'documentation']","['IS', 'GB']",NaN,tt6330514,6.3,2304.0,3.187,6.100
4720,ts271437,Money Heist (Korean Remake),SHOW,A story of genius strategists and robbers with...,2022,TV-MA,70,"['action', 'drama', 'thriller', 'crime']",['KR'],1.0,tt13696452,5.2,5142.0,904.326,7.998
211,tm314602,Dushmani,MOVIE,In order to settle personal scores; two gang l...,1995,NaN,156,"['romance', 'action', 'drama']",['IN'],NaN,tt0112916,5.0,249.0,2.045,7.500
3331,tm919864,The Disciple,MOVIE,"Self-doubt, sacrifice and struggle converge in...",2020,NaN,128,"['drama', 'music']",['IN'],NaN,tt11423784,7.1,2567.0,3.490,6.800
2980,tm368284,Arango y Sanint: Ríase el show,MOVIE,Colombian stand-up comedians Antonio Sanint an...,2018,NaN,62,['comedy'],['CO'],NaN,tt7606464,4.0,42.0,1.218,4.000
2535,tm286532,Rock my Heart,MOVIE,17-year-old Jana has a congenital heart defect...,2017,NaN,105,"['drama', 'family', 'action', 'european']",['DE'],NaN,tt5870322,6.5,993.0,7.023,7.200
2722,tm351201,In the Family,MOVIE,"The neurotic Fikret and tavern singer Solmaz, ...",2017,PG-13,124,['comedy'],['TR'],NaN,tt7642818,7.7,23542.0,6.241,6.784
2485,ts268283,RIDE ON TIME,SHOW,NaN,2018,TV-PG,22,[],['JP'],3.0,NaN,NaN,NaN,1.328,NaN
2797,ts84372,YG Future Strategy Office,SHOW,"Kpop star Seungri, BIGBANG youngest member, tr...",2018,TV-MA,26,['comedy'],['KR'],1.0,tt8129694,6.7,209.0,2.519,7.100
617,tm85369,Hattrick,MOVIE,"Despite cricket fever running wild, there are ...",2007,PG,110,"['drama', 'comedy', 'romance', 'sport']",['IN'],NaN,tt0979891,4.3,495.0,2.115,5.000


从抽样的10行数据来看，'original_titles'中的数据不符合“每行是一个观察值，每列是一个变量”。\
具体来看，每行是关于某影视作品的具体信息，每列是影视作品相关的各个变量，而`genres`和`production_countries`的变量中包含多个值，应当进行拆分

>先提取任意一个genres变量的值进行观察。

In [6]:
cleaned_titles['genres'][1]

"['drama', 'crime']"

虽然genres表示形式是列表，但其实际类型并非 字符串列表，而是 字符串，无法直接用value_counts统计各个值出现的次数。\
我们可以使用Python内置的eval函数，它可以**把字符串转换成表达式**，所以可以帮我们把表示**列表的字符串**转换成**列表**本身。
>`eval(expression[, globals[, locals]])` \
>eval() 函数将字符串 expression 解析为 Python 表达式，并在指定的命名空间中执行它。\
>eval() 函数将字符串转换为相应的对象，并返回表达式的结果。

In [7]:
cleaned_titles['genres']=cleaned_titles['genres'].apply(lambda s : eval(s))

In [8]:
cleaned_titles['genres'][1]

['drama', 'crime']

>转换为列表后，就能用DataFrame的explode方法，把那个列的列表值拆分成单独的行。

In [9]:
cleaned_titles=cleaned_titles.explode('genres')
cleaned_titles['genres'][1]

1    drama
1    crime
Name: genres, dtype: object

>production_countries也如同genres操作

In [10]:
cleaned_titles['production_countries'][1]

1    ['US']
1    ['US']
Name: production_countries, dtype: object

In [11]:
cleaned_titles['production_countries']=cleaned_titles['production_countries'].apply(lambda s : eval(s))
cleaned_titles['production_countries'][1]

1    [US]
1    [US]
Name: production_countries, dtype: object

In [12]:
cleaned_titles=cleaned_titles.explode('production_countries')
cleaned_titles['production_countries'][1]

1    US
1    US
Name: production_countries, dtype: object

>在处理完cleaned_titles的结构性问题后，查看cleaned_credits。

In [18]:
cleaned_credits.sample(10)

,person_id,id,name,character,role
49927,99858,tm845437,Phil Donahue,Self (archive footage),ACTOR
20575,96009,tm239727,Tom Ford,NaN,DIRECTOR
4592,1514718,tm91952,Shannon Todd,Junior Plastic,ACTOR
45622,2139046,tm460948,Kipp Glass,Maximal Patron,ACTOR
15304,64512,tm39606,Tinnu Anand,NaN,ACTOR
18586,135621,ts20781,Tae In-ho,Sung Joon-sik,ACTOR
20700,6600,tm140391,Garrett Hedlund,James Hook,ACTOR
23572,849741,tm253080,Veruca James,Self,ACTOR
16910,16207,tm161282,Billy Eichner,New York Reporter (voice),ACTOR
51219,725651,tm885775,Heidi Gardner,Jenny,ACTOR


从抽样的10行数据来看，`original_credits`中的数据符合“每行是一个观察值，每列是一个变量”。\
具体来看，每行是关于某演员的具体信息，每列是演员相关的各个变量。因此，`original_credits`不存在结构性问题。

### 2.内容

#### 2.1 缺失数据

In [15]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

从输出结构来看，数据共有17818条观察值，而`original_titles`中的`title`,`description`,`age_certification`,`genres`,`seasons`,`imdb_id`,`imdb_score`,`imdb_votes`,`imdb_popularity`,`tmdb_score`变量中存在缺失值，将在后续进行评估。\
此外，release_year是发布年份，数据类型应为日期，应当进行数据类型的转换。

>对release_year进行数据类型转换，转换为日期。

In [39]:
cleaned_titles['release_year']=pd.to_datetime(cleaned_titles['release_year'],format='%Y')
cleaned_titles['release_year']

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

由于 title：影视作品标题，description：简短描述，age_certification：适龄认证，seasons：如果是电视剧，则是季数，imdb_id：IMDB的ID，imdb_votes：IMDB的投票数，tmdb_popularity：TMDB的流行度，tmdb_score：TMDB的评分并不影响我们挖掘各个流派中的高IMDB评分作品演员。

所以变量title,description,age_certification,seasons,imdb_id,imdb_votes,imdb_popularity,tmdb_score存在空缺的观察值可以保留。

但imdb_score和genres ，即IMDB评分和流派类型，和我们后续要做的分析息息相关。

>先提取genres变量缺失的观察值

In [41]:
cleaned_titles[cleaned_titles['genres'].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
212,tm255589,One Last Shot,MOVIE,"In this low-budget short film, two best buddie...",1998-01-01,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,1.890,5.2
619,tm341561,Like Twenty Impossibles,MOVIE,Occupied Palestine: A serene landscape now poc...,2003-01-01,NaN,16,NaN,PS,NaN,NaN,NaN,NaN,0.812,6.5
632,ts86241,Le Robe De Mariage Des Cieux,SHOW,It was with much difficulty that Ai Qing was a...,2004-01-01,TV-MA,63,NaN,NaN,1.0,NaN,NaN,NaN,0.600,NaN
636,tm404676,To and from New York,MOVIE,"While covering a story in New York City, a Sea...",2006-01-01,NaN,82,NaN,US,NaN,NaN,NaN,NaN,1.401,5.8
637,tm89054,Osuofia in London 2,MOVIE,Osuofia return to his Nigerian village with a ...,2004-01-01,NaN,72,NaN,XX,NaN,NaN,NaN,NaN,1.091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,tm1040959,The Circle: The Afterparty,MOVIE,Stars of The Circle drop by to discuss Season ...,2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,1.882,10.0
5802,ts302434,Plastic Cup Boyz: Laughing My Mask Off!,SHOW,Comedy collective The Plastic Cup Boyz pour ou...,2021-01-01,NaN,33,NaN,NaN,1.0,NaN,NaN,NaN,0.683,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


由于缺失分析所需的核心数据genres，会对后续的分析产生影响。
>我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [47]:
cleaned_titles=cleaned_titles.dropna(subset='genres')
cleaned_titles['genres'].isnull().sum()

0

>然后提取先提取imdb_score观察

In [48]:
cleaned_titles[cleaned_titles['imdb_score'].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,tm1094060,My Heroes Were Cowboys,MOVIE,Robin Wiltshire's painful childhood was rescue...,2021-01-01,PG,23,documentation,US,NaN,tt15084326,NaN,NaN,3.145,7.7
5791,tm1047429,Alan Saldaña: Locked Up,MOVIE,"Mexican comedian Alan Saldaña is back, poking ...",2021-01-01,NaN,49,comedy,NaN,NaN,NaN,NaN,NaN,6.670,6.0
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,comedy,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN


In [50]:
cleaned_titles[(cleaned_titles['imdb_score'].isnull())&(cleaned_titles['imdb_votes'].notna())]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


筛选出结果数量为0条，说明imdb_score值缺失的值也不具备有效的imdb_votes值。\
imdb_score：IMDB的评分\
imdb_votes：IMDB的投票数\
都是进行后续高评分作品演员分析的重要变量，如果它们同时缺失/无效，我们认为数据无法提供有效含义，因此这些数据后续可以删除。

>由于缺失分析所需的核心数据`imdb_score`，我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [53]:
cleaned_titles=cleaned_titles.dropna(subset='imdb_score')
cleaned_titles['imdb_score'].isnull().sum()

0

>查看cleaned_credits。

In [19]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


从输出结构来看，数据共有77801条观察值，而original_titles中的character变量中存在缺失值，将在后续进行评估。\
此外，person_id是演职员ID，数据类型应为字符串，应当进行数据类型的转换。

In [76]:
#cleaned_credits['person_id'].astype(str)

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

character：角色姓名，在后续的分析中并不影响我们挖掘各个流派中的高IMDB评分作品演员，因此可以保留。

#### 2.2 重复数据

根据数据变量的含义以及内容来看，cleaned_titles里不应该存在每个变量值都相同的观察值，因此查看是否存在重复值。

>查看cleaned_titles数据表是否存在重复值。

In [55]:
cleaned_titles.duplicated().sum()

0

$输出结果为0，说明不存在重复值。$

>查看cleaned_credits数据表是否存在重复值。

In [78]:
cleaned_credits.duplicated().sum()

0

$输出结果为0，说明不存在重复值。$

#### 2.3 不一致数据

针对cleaned_titles，不一致数据可能存在于genres和production_countries变量中，我们将查看是否存在多个不同值指代同一流派，以及多个不同值指代同一国家的情况。

In [60]:
cleaned_titles['genres'].value_counts()

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

从上面看出，genres列里并不存在不一致数据，各个值都在指代不同的流派。但是里面还存在空字符串表示的流派，并非有效数据，因此可以进行删除。

In [63]:
cleaned_titles = cleaned_titles.query('genres != ""')
cleaned_titles.query('genres == ""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


接下来，针对`production_countries`列也是一样的流程，利用`value_counts`方法，得到`production_countries`的列表里面各个值的出现次数。

In [64]:
cleaned_titles['production_countries'].value_counts()

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
GT       1
CU       1
LK       1
NP       1
FO       1
Name: count, Length: 108, dtype: int64

由于`value_counts`执行结果中有太多值，Pandas只会默认显示开头和结尾的一些值。\
要完整展示结果，可以把`display.max_rows`设置为`None`，即取消展示行数上限。\
但因为我们只是在当前调用`value_counts`时才需要看完整结果，所以可以结合`option_context`，只更改临时上限。

In [66]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

从以上输出结果来看，出品国家都用两位的国家代码来表示，除了里面存在一个的`Lebanon`值。\
`Lebanon`的国家代码是`LB`，出现了70次，说明此处数据不一致。\
`LB`和`Lebanon`都在表示同一国家，需要进行统一。

>把`cleaned_titles`里，`production_countries`的`"LB"`和`"Lebanon"`统一为`LB`，并检查替换后是否还存在`"LB"`：

In [71]:
cleaned_titles['production_countries']=cleaned_titles['production_countries'].replace('Lebanon','LB')

In [72]:
# 检查"Lebanon"是否还存在
with pd.option_context('display.max_rows', None):
    print(cleaned_titles.explode('production_countries')['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


另外，里面还存在空字符串表示的国家代码，并非有效数据。但由于出品国家并非分析所需的关键信息，所以可以保留出品国家为空的观察值。

针对original_credits，不一致数据可能存在于role中，我们将查看是否存在多个不同值指代同一演职员类型的情况。

In [79]:
cleaned_credits['role'].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从以上输出结果来看，role只有两种可能的值，ACTOR或DIRECTOR，不存在不一致数据。\
我们可以把这列的类型转换为Category，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [83]:
cleaned_credits['role']=cleaned_credits['role'].astype('category')
cleaned_credits['role']

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

#### 2.4 无效或错误数据

In [73]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,2015-11-14 22:42:51.974072064,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,2018-01-01 00:00:00,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,2020-01-01 00:00:00,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


从以上统计信息来看，original_titles里不存在脱离现实意义的数值。

original_credits由于不包含表示数值含义的变量，因此无需用describe检查。

# 整理数据

In [13]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,NaN,US,NaN,NaN,NaN,NaN,1.296,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


In [14]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


对数据的整理，与分析方向紧密相关。此次数据分析目标是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

那为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。

In [28]:
titels_with_credits=pd.merge(cleaned_titles,cleaned_credits,on='id',how='inner')
titels_with_credits.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


由于我们只对挖掘演员的参演作品口碑感兴趣，导演不在我们的分析范围内。因此根据role，筛选出类型为ACTOR的观察值，供后续分析。

>筛选出“role”为“actor”的值

In [31]:
titels_with_credits=titels_with_credits.query('role =="ACTOR" ')
titels_with_credits.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据流派和演员进行分组。

对演员进行分组的时候，选择的是用`person_id`而不是`name`变量，原因是名字容易出现错拼或者重名的情况，演职员ID会比演员姓名更加准确地反映是哪位演员。

>根据流派，对演员进行分组

In [37]:
groupby_genres_and_person_id =titels_with_credits.groupby(['genres','person_id'])

分组后，我们只需要对imdb_score的值进行聚合计算，因此只提取imdb_score变量，然后调用mean，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分。

>计算各个流派中每位演员参演作品的平均IMDB评分

In [38]:
imdb_score_groupby_genres_and_person_id=groupby_genres_and_person_id['imdb_score'].mean()
imdb_score_groupby_genres_and_person_id

genres   person_id
action   45           5.0
         48           5.4
         51           6.4
         53           6.8
         54           5.3
                     ... 
western  2353339      6.9
         2370848      6.1
         2398539      3.8
         2406218      6.0
         2408082      7.3
Name: imdb_score, Length: 172860, dtype: float64

>我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [40]:
imdb_score_groupby_genres_and_person_id_df=imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,45,5.0
1,action,48,5.4
2,action,51,6.4
3,action,53,6.8
4,action,54,5.3
...,...,...,...
172855,western,2353339,6.9
172856,western,2370848,6.1
172857,western,2398539,3.8
172858,western,2406218,6.0


现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。

但我们当前可以继续做一些数据整理，比如对上面的结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。

>要得到这一结果，我们需要再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。

In [43]:
genres_max_scores=imdb_score_groupby_genres_and_person_id_df.groupby('genres')['imdb_score'].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

在我们知道最高分后，可以把以上结果和之前得到的`imdb_score_groupby_genres_and_person_id_df`再次进行连接，得到最高分对应的各个演员ID是什么，也就是这个最高平均分是哪位演员拿到的。

In [45]:
genres_max_score_with_person_id=pd.merge(imdb_score_groupby_genres_and_person_id_df,genres_max_scores,on=['genres','imdb_score'])
genres_max_score_with_person_id

,genres,person_id,imdb_score
0,action,1303,9.3
1,action,12790,9.3
2,action,21033,9.3
3,action,86591,9.3
4,action,336830,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


从以上结果可以看出，最高分对应的演员不一定只有一位，可能有多位演员的平均得分相同。

>为了得到演员ID所对应的演员名字，我们可以和cleaned_credits这个DataFrame进行连接。这个DataFrame还有其它列,我们只需要得到person_id和name的对应，所以可以先提取出那两列，并把重复行删除。

In [48]:
actor_id_with_names=cleaned_credits[['person_id','name']].drop_duplicates()
actor_id_with_names.head(10)

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
5,32267,Peter Boyle
6,519612,Leonard Harris
7,29068,Diahnne Abbott
8,519613,Gino Ardito
9,3308,Martin Scorsese


>下一步就可以把actor_id_with_names与前面得到的genres_max_score_with_person_id进行连接，增加name变量，从而展示平均评分最高的演员名字。

In [51]:
genres_max_score_with_actor_name =pd.merge(genres_max_score_with_person_id,actor_id_with_names,on='person_id')
genres_max_score_with_actor_name 

,genres,person_id,imdb_score,name
0,action,1303,9.3,Jessie Flower
1,animation,1303,9.3,Jessie Flower
2,family,1303,9.3,Jessie Flower
3,fantasy,1303,9.3,Jessie Flower
4,scifi,1303,9.3,Jessie Flower
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


为了把相同流派都排序在一起，我们还可以用`sort_values`方法，把结果里面的行根据`genres`进行排序，然后用`reset_index`把索引重新排序。

索引重新排序后，DataFrame会多出`index`一列，我们可以再把`index`列进行删除。

In [54]:
genres_max_score_with_actor_name.sort_values('genres')

,genres,person_id,imdb_score,name
0,action,1303,9.3,Jessie Flower
12,action,86591,9.3,Cricket Leigh
7,action,21033,9.3,Zach Tyler
5,action,12790,9.3,Olivia Hack
17,action,336830,9.3,André Sogliuzzo
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka
132,western,22311,8.9,Koichi Yamadera


In [53]:
genres_max_score_with_actor_name.sort_values('genres').reset_index()

,index,genres,person_id,imdb_score,name
0,0,action,1303,9.3,Jessie Flower
1,12,action,86591,9.3,Cricket Leigh
2,7,action,21033,9.3,Zach Tyler
3,5,action,12790,9.3,Olivia Hack
4,17,action,336830,9.3,André Sogliuzzo
...,...,...,...,...,...
131,131,war,826547,8.8,Yuto Uemura
132,133,western,28166,8.9,Megumi Hayashibara
133,134,western,28180,8.9,Unsho Ishizuka
134,132,western,22311,8.9,Koichi Yamadera


In [56]:
genres_max_score_with_actor_name=genres_max_score_with_actor_name.sort_values('genres').reset_index().drop('index',axis=1)
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,1303,9.3,Jessie Flower
1,action,86591,9.3,Cricket Leigh
2,action,21033,9.3,Zach Tyler
3,action,12790,9.3,Olivia Hack
4,action,336830,9.3,André Sogliuzzo
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28166,8.9,Megumi Hayashibara
133,western,28180,8.9,Unsho Ishizuka
134,western,22311,8.9,Koichi Yamadera
